<a href="https://colab.research.google.com/github/allispaul/audiobot/blob/main/AG_fmasmall_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [83]:
import os
import multiprocessing
import ast
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm
from pathlib import Path

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score

from xgboost import XGBClassifier

In [96]:
# Model assessment function
def model_assess(model, title="Default"):
    model.fit(X_train,y_train)
    preds = model.predict(X_val)
    print('Accuracy', title, ":", round(accuracy_score(y_val,preds), 5), "\n")
    print('F-Score', title, ":", round(f1_score(y_val,preds, average="macro"),5), "\n")

In [16]:
# get track metadata
METADATA_DIR = Path("/content/drive/MyDrive/Audiobots/Data/fma_metadata")

tracks_info = pd.read_csv(METADATA_DIR / 'tracks.csv', index_col=0, header=[0, 1])
tracks_info_subset = tracks_info.copy()
tracks_info_subset.columns = ['_'.join(col).strip() for col in tracks_info_subset.columns.values]
tracks_info_subset = tracks_info_subset[["set_split","set_subset","track_duration","track_genre_top"]]

## FMA-Small

The section below gets track information from fma metadata, imports fma_small, merges and cleans the data to get it ready for XGBoost.

In [19]:
# Track info for fma_small
tracks_small = tracks_info_subset[tracks_info_subset["set_subset"]=="small"]
tracks_small.shape, tracks_small["track_duration"].describe()

In [36]:
# Track info for fma_medium
tracks_med = tracks_info_subset[tracks_info_subset["set_subset"]=="medium"]
tracks_med.shape, tracks_med["track_duration"].describe()

((17000, 4),
 count    17000.000000
 mean       230.409824
 std        103.986197
 min         60.000000
 25%        156.000000
 50%        213.000000
 75%        285.000000
 max        600.000000
 Name: track_duration, dtype: float64)

In [86]:
# Import fma_small
data_path = "/content/drive/MyDrive/Audiobots/Data/fmasmall_features_3sec.csv"

#fma_small = pd.read_csv(data_path,header=None)
fma_small = pd.read_csv(data_path,index_col=0,header=[0,1,2,3])
fma_small.columns = ['_'.join(col).strip() for col in fma_small.columns.values]

In [87]:
# Merge track info with features
# 6400 training, 800 validation, 800 test set
# FMA_small is balanced - each genre has 1000 observations with 8 genres
fma_small_full = tracks_small.merge(fma_small,left_index=True,right_index=True)
fma_small_full.head()

,set_split,set_subset,track_duration,track_genre_top,chroma_cens_max_01_pos_01,chroma_cens_max_01_pos_02,chroma_cens_max_01_pos_03,chroma_cens_max_01_pos_04,chroma_cens_max_01_pos_05,chroma_cens_max_01_pos_06,...,zcr_std_01_pos_01,zcr_std_01_pos_02,zcr_std_01_pos_03,zcr_std_01_pos_04,zcr_std_01_pos_05,zcr_std_01_pos_06,zcr_std_01_pos_07,zcr_std_01_pos_08,zcr_std_01_pos_09,zcr_std_01_pos_10
track_id,,,,,,,,,,,,,,,,,,,,,
2,training,small,168,Hip-Hop,0.704124,0.670529,0.731929,0.699341,0.558846,0.546863,...,0.058834,0.086403,0.061894,0.071970,0.069562,0.050927,0.044658,0.055987,0.066270,0.089225
5,training,small,206,Hip-Hop,0.386654,0.396511,0.430391,0.420343,0.459678,0.469663,...,0.039076,0.045397,0.051211,0.040011,0.036974,0.029119,0.041461,0.033762,0.072484,0.060742
10,training,small,161,Pop,0.413630,0.393576,0.306028,0.380294,0.286272,0.330367,...,0.021721,0.034129,0.020003,0.018932,0.016606,0.015600,0.018541,0.018028,0.018082,0.014767
140,training,small,253,Folk,0.474108,0.244633,0.398619,0.297416,0.225072,0.398411,...,0.013485,0.026601,0.033925,0.023193,0.039791,0.038043,0.039282,0.007803,0.014162,0.064145
141,training,small,182,Folk,0.519412,0.436285,0.505933,0.355401,0.042728,0.354588,...,0.059952,0.015979,0.045776,0.044014,0.021849,0.030574,0.014621,0.011454,0.022677,0.026572


In [88]:
# drop index
fma_small_full = fma_small_full.reset_index(drop=True)
fma_small_full.head()

,set_split,set_subset,track_duration,track_genre_top,chroma_cens_max_01_pos_01,chroma_cens_max_01_pos_02,chroma_cens_max_01_pos_03,chroma_cens_max_01_pos_04,chroma_cens_max_01_pos_05,chroma_cens_max_01_pos_06,...,zcr_std_01_pos_01,zcr_std_01_pos_02,zcr_std_01_pos_03,zcr_std_01_pos_04,zcr_std_01_pos_05,zcr_std_01_pos_06,zcr_std_01_pos_07,zcr_std_01_pos_08,zcr_std_01_pos_09,zcr_std_01_pos_10
0,training,small,168,Hip-Hop,0.704124,0.670529,0.731929,0.699341,0.558846,0.546863,...,0.058834,0.086403,0.061894,0.071970,0.069562,0.050927,0.044658,0.055987,0.066270,0.089225
1,training,small,206,Hip-Hop,0.386654,0.396511,0.430391,0.420343,0.459678,0.469663,...,0.039076,0.045397,0.051211,0.040011,0.036974,0.029119,0.041461,0.033762,0.072484,0.060742
2,training,small,161,Pop,0.413630,0.393576,0.306028,0.380294,0.286272,0.330367,...,0.021721,0.034129,0.020003,0.018932,0.016606,0.015600,0.018541,0.018028,0.018082,0.014767
3,training,small,253,Folk,0.474108,0.244633,0.398619,0.297416,0.225072,0.398411,...,0.013485,0.026601,0.033925,0.023193,0.039791,0.038043,0.039282,0.007803,0.014162,0.064145
4,training,small,182,Folk,0.519412,0.436285,0.505933,0.355401,0.042728,0.354588,...,0.059952,0.015979,0.045776,0.044014,0.021849,0.030574,0.014621,0.011454,0.022677,0.026572


In [93]:
# Prepare data for XGBoost
X_train = fma_small_full[fma_small_full["set_split"]=="training"]
X_train = X_train.drop(["set_split","set_subset","track_duration","track_genre_top"], axis=1)
y_train = fma_small_full[fma_small_full["set_split"]=="training"]["track_genre_top"]
X_val = fma_small_full[fma_small_full["set_split"]=="validation"]
X_val = X_val.drop(["set_split","set_subset","track_duration","track_genre_top"], axis=1)
y_val = fma_small_full[fma_small_full["set_split"]=="validation"]["track_genre_top"]

In [94]:
# Encode DV
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [95]:
# Normalize X features
cols = X_train.columns

scaler = MinMaxScaler()

# Training IV features
X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled, columns = cols)

# Test IV features
X_test_scaled = scaler.transform(X_val)
X_test = pd.DataFrame(X_test_scaled, columns=cols)

In [98]:
# XGBoost with GPUs

xgb_gpu = XGBClassifier(n_estimators=1000, learning_rate=0.05, tree_method="gpu_hist")
model_assess(xgb_gpu, "Cross Gradient Booster with GPUs")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:34:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Accuracy Cross Gradient Booster with GPUs : 0.14 

F-Score Cross Gradient Booster with GPUs : 0.08754 



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:39:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [02:39:43] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


## FMA-Medium

The section below gets track information from fma metadata, imports fma_medium, merges and cleans the data to get it ready for XGBoost.

In [ ]:
# Import fma_medium
data_path_med = "/content/drive/MyDrive/Audiobots/Data/fma_features_3sec.csv"

#fma_small = pd.read_csv(data_path,header=None)
fma_med = pd.read_csv(data_path_med,index_col=0,header=[0,1,2,3])
fma_med.columns = ['_'.join(col).strip() for col in fma_med.columns.values]

In [ ]:
# Merge track info with features
# X training, X validation, X test set
# FMA_medium is highly imbalanced - 16 genres,
fma_med_full = tracks_med.merge(fma_med,left_index=True,right_index=True)
fma_med_full.head()

In [ ]:
# drop index
fma_med_full = fma_med_full.reset_index(drop=True)
fma_med_full.head()

In [ ]:
# Prepare data for XGBoost
X_train = fma_med_full[fma_med_full["set_split"]=="training"]
X_train = X_train.drop(["set_split","set_subset","track_duration","track_genre_top"], axis=1)
y_train = fma_med_full[fma_med_full["set_split"]=="training"]["track_genre_top"]
X_val = fma_med_full[fma_med_full["set_split"]=="validation"]
X_val = X_val.drop(["set_split","set_subset","track_duration","track_genre_top"], axis=1)
y_val = fma_med_full[fma_med_full["set_split"]=="validation"]["track_genre_top"]

In [ ]:
# Encode DV
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [ ]:
# Normalize X features
cols = X_train.columns

scaler = MinMaxScaler()

# Training IV features
X_train_scaled = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train_scaled, columns = cols)

# Test IV features
X_test_scaled = scaler.transform(X_val)
X_test = pd.DataFrame(X_test_scaled, columns=cols)

In [ ]:
# XGBoost with GPUs

xgb_gpu = XGBClassifier(n_estimators=1000, learning_rate=0.05, tree_method="gpu_hist")
model_assess(xgb_gpu, "Cross Gradient Booster with GPUs")